In [ ]:
import sys
sys.path.insert(0,'..')
from datagenerator import jet_data_generator 
import time
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
import os
import pandas as pd

sys.path.insert(0,'../training')
from manifoldembedder import JetDataset, JetPredictDataset
from torch.utils.data import DataLoader, random_split

# Pair Jet Data Generation

In [ ]:
savedir   = '/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/'
rawjetdir = '/nobackup/users/sangeon/datasets/jetGridpack/rawjets/'
type_arr  = [['QCD','QCD',''],['2p','twoprong','_80'],['2p','twoprong','_400'],['3p','threeprong','_80'],['3p','threeprong','_400']]
len_train = 1000000
len_half  = 500000
start = time.time()
for i in range(0,2000):
    np.random.seed()
    type_1 = type_arr[np.random.randint(0,5)]
    np.random.seed()
    type_2 = type_arr[np.random.randint(0,5)]

    df1 = pd.read_hdf(os.path.join(rawjetdir,type_1[0],f'{type_1[1]}{type_1[2]}.h5'))
    df2 = pd.read_hdf(os.path.join(rawjetdir,type_2[0],f'{type_2[1]}{type_2[2]}.h5'))
    A = df1.values
    B = df2.values
    
    np.random.seed()
    perm = np.random.permutation(len_train)
    print(os.path.join(rawjetdir,type_1[0],f'{type_1[1]}{type_1[2]}.h5'))
    print(os.path.join(rawjetdir,type_2[0],f'{type_2[1]}{type_2[2]}.h5'))
    
    train_set = JetDataset(False, None, False, A[perm[:10000]],B[perm[len_half:len_half+10000]],16)
    torch.save(train_set, os.path.join(savedir,f'Gridpack_jetpair_train_16part_{i}.pt'))
    
end = time.time()
print('total time', end-start)

In [ ]:
idx_list = []
for i in range(5):
    for j in range(5):
        idx_list.append([i,j])
        


savedir   = '/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/'
rawjetdir = '/nobackup/users/sangeon/datasets/jetGridpack/rawjets/'
type_arr  = [['QCD','QCD',''],['2p','twoprong','_80'],['2p','twoprong','_400'],['3p','threeprong','_80'],['3p','threeprong','_400']]
len_val = 200000
len_half  = 100000
start = time.time()
for idx in range(25):
    np.random.seed()

    type_1 = type_arr[idx_list[idx][0]]
    type_2 = type_arr[idx_list[idx][1]]
    print(type_1, type_2)
    
    df1 = pd.read_hdf(os.path.join(rawjetdir,type_1[0],f'{type_1[1]}{type_1[2]}_val.h5'))
    df2 = pd.read_hdf(os.path.join(rawjetdir,type_2[0],f'{type_2[1]}{type_2[2]}_val.h5'))
    A = df1.values
    B = df2.values
    
    np.random.seed()
    perm = np.random.permutation(len_val)

    print(os.path.join(rawjetdir,type_1[0],f'{type_1[1]}{type_1[2]}_val.h5'))
    print(os.path.join(rawjetdir,type_2[0],f'{type_2[1]}{type_2[2]}_val.h5'))
    
    val_set = JetDataset(False, None, False, A[perm[:len_half]],B[perm[len_half:]],16)
    torch.save(val_set, os.path.join(savedir,f'Gridpack_jetpair_val_16part_{idx}.pt'))
    
end = time.time()
print('total time', end-start)

# Predict, Test Dataset

In [ ]:
rawjetdir = '/nobackup/users/sangeon/datasets/jetGridpack/rawjets/'
type_arr  = [['QCD','QCD',''],['2p','twoprong','_80_predict'],['2p','twoprong','_400_predict'],['3p','threeprong','_80_predict'],['3p','threeprong','_400_predict'],['4p','fourprong','_170'],['4p','fourprong','_400']]
set_list = []
name_list = ['QCD','2p25','2p170','3p25','3p170','4p170','4p400']

results = np.empty((0,54))
predict_label = np.empty((0))
for i, (name, jettype) in enumerate(zip(name_list, type_arr)):
    df = pd.read_hdf(os.path.join(rawjetdir,jettype[0],f'{jettype[1]}{jettype[2]}.h5'))
    print(df.values.shape)
    data = df.values[:,:54]
    data = data[:100000]
    results = np.vstack([results, data])
    print(df.values.shape[0])
    print(name, jettype)
    predict_label = np.concatenate([predict_label, np.repeat(i, data.shape[0])])

In [ ]:
predict_set = JetPredictDataset(False,None,False, results,predict_label,16)
torch.save(predict_set, f'/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/Gridpack_jetpair_secondensemble_predict.pt')